# Raster masking

This notebook contains simple code for applying a limits (clipping) mask to a series of rasters. Effectively it just takes a path to the mask, which must contain values 0 and 1 only, and multiplies all the incoming rasters by it. All the rasters, including the mask, must be of the same size / extent / resolution!

In [5]:
from osgeo import gdal
import numpy as np
import os
import glob

In [1]:
from General_Raster_Funcs.TiffManagement import SaveLZWTiff

In [ ]:
#inFiles = glob.glob(r'\\129.67.26.176\map_data\ROAD-MAP\data\Website_GIS\Pf_Mapping\Served_Rasters\Interventions\ITN\*.tif')
inFiles = glob.glob(r'\\129.67.26.176\map_data\ROAD-MAP\data\Website_GIS\Pf_Mapping\Served_Rasters\Incidence_actual_annual_means\*.tif')

In [7]:
maskRaster = r'C:\Temp\dataprep\Pf_Limits_2016_NoAPI.tif'
maskData = gdal.Open(maskRaster)
maskBnd = maskData.GetRasterBand(1)
maskArr = maskBnd.ReadAsArray()

In [ ]:
for f in inFiles:
    d = gdal.Open(f)
    b = d.GetRasterBand(1)
    dt = b.DataType
    arr = b.ReadAsArray()
    assert arr.shape == maskArr.shape
    ndv = b.GetNoDataValue()
    if ndv is None:
        ndv = -9999
        
    if np.all(arr==0):
        arr[:]=ndv
    else:
        arr[maskArr!=1]=ndv
    
    outFN = f.replace(".tif", ".stable.tif")
    
    
    SaveLZWTiff(arr, ndv, d.GetGeoTransform(), d.GetProjection(), 
                os.path.dirname(f), 
                os.path.basename(outFN))